In [1]:
pip install requests


In [1]:
import requests
import csv
from datetime import datetime


channel_id = "2733660"
read_api_key = "3FL4Z3Z338O5XM47"
url = f"https://api.thingspeak.com/channels/{channel_id}/feeds.json?api_key={read_api_key}&results=100"  # Get the last 100 entries


response = requests.get(url)
data = response.json()


filename = f"thingspeak_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"


with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["Timestamp", "RMS", "Mean Frequency", "Variance", "Amplitude"])


    for entry in data['feeds']:
        timestamp = entry['created_at']
        rms = entry['field1']
        mean_frequency = entry['field2']
        variance = entry['field3']
        amplitude = entry['field4']
        writer.writerow([timestamp, rms, mean_frequency, variance, amplitude])

print(f"Data saved to {filename}")


Data saved to thingspeak_data_20241107_194841.csv


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
import pandas as pd


file_path = '/content/Dataset_TENS_with_status.csv'
data = pd.read_csv(file_path)


X = data[['RMS', 'Mean Frequency', 'Variance', 'Amplitude']]
y_status = data['status']
y_pain = data['pain_level']


X_train, X_test, y_status_train, y_status_test, y_pain_train, y_pain_test = train_test_split(
    X, y_status, y_pain, test_size=0.3, random_state=42
)


rf_status = RandomForestClassifier(n_estimators=100, random_state=42)
rf_pain = RandomForestClassifier(n_estimators=100, random_state=42)


rf_status.fit(X_train, y_status_train)
rf_pain.fit(X_train, y_pain_train)


y_status_pred = rf_status.predict(X_test)
y_pain_pred = rf_pain.predict(X_test)


status_accuracy = accuracy_score(y_status_test, y_status_pred)
print(f"Cramping Status Prediction Accuracy: {status_accuracy * 100:.2f}%")
print("Cramping Status Classification Report:")
print(classification_report(y_status_test, y_status_pred, target_names=['Not Cramping', 'Cramping']))


if y_pain.dtype == 'int' or y_pain.dtype == 'object':
    pain_accuracy = accuracy_score(y_pain_test, y_pain_pred)
    print(f"Pain Level Prediction Accuracy: {pain_accuracy * 100:.2f}%")
    print("Pain Level Classification Report:")
    print(classification_report(y_pain_test, y_pain_pred))
else:
    pain_mse = mean_squared_error(y_pain_test, y_pain_pred)
    print(f"Pain Level Prediction Mean Squared Error: {pain_mse:.2f}")


Cramping Status Prediction Accuracy: 89.29%
Cramping Status Classification Report:
              precision    recall  f1-score   support

Not Cramping       0.91      0.95      0.93        22
    Cramping       0.80      0.67      0.73         6

    accuracy                           0.89        28
   macro avg       0.86      0.81      0.83        28
weighted avg       0.89      0.89      0.89        28

Pain Level Prediction Accuracy: 89.29%
Pain Level Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.92      0.92      0.92        12
           3       0.90      0.90      0.90        10
           4       0.00      0.00      0.00         0
           5       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         3

    accuracy                           0.89        28
   macro avg       0.64      0.64      0.64        28
weighted avg       0.89      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_